# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
import pandas as pd
import numpy as np
from emoji import UNICODE_EMOJI

df = pd.read_excel('tweets_netflix_treinamento.xlsx')

df.replace(regex=['\n', '\t'], value = '', inplace = True) #corrigindo separação entre enter e tab - pt.1

In [2]:
tweets = []

remover = ['?', '!', ',', ';', '.', '”', '“', ')',
           '(', '*', '...', ':', '…', '{', '}', '"',
           '"', '=', "'", '•', '|', '-', '/'] #caracteres que devem ser removídos
for i in range(len(df['Treinamento'])):
    m = ''
    tweet = df['Treinamento'][i]
    for letra in tweet:
        if letra in UNICODE_EMOJI:
            m += ' ' + letra  + ' ' #corrigindo espaçamento entre caracteres - pt.2
        elif letra not in remover:
            m += letra
    tweets.append(m)

In [3]:
tweets_final = []
words = ['rt', 'a', 'is', 'me', 'in', 'netflix', 'as', 'it',
         'we', 'us', 'of', 'i', 'to', 'the', 'by', 'im', 'so',
         'are', 'or', 'has', 'have', 'their', 'that', 'else',
         'on', 'be', 'his', 'our', 'you', 'my', 'its', 'for',
         "it’s", 'this', 'if']
for i in tweets:
    post = i.split()
    palavras = ''
    for e in post:
        #removendo links, hashtags, @, e demais termos/palavras inúteis
        if e[0:4] != 'http' and e[0:1] != '@' and e[:1] != '#' and e not in words:
            palavras += '' + e + ' '
        elif e in UNICODE_EMOJI:
            palavras += '' + e + ''
            
    tweets_final.append(palavras)

In [4]:
data = pd.DataFrame(tweets_final, columns=['Treinamento'])
data = data.join(df['Relevância'], how='inner')

data.head()

In [5]:
tudo = []

for tweet in data['Treinamento']:
    a = tweet.split(' ')
    for palavra in a:
        if palavra not in tudo:
            tudo.append(palavra)
            
conta_relevante = 0
conta_irrelevante = 0

for i in range(len(data['Treinamento'])):
    a = data["Treinamento"][i].split(" ")
    if data['Relevância'][i] == 1:
        for palavra in a:
            conta_relevante += 1
    else:
        for palavra in a:
            conta_irrelevante += 1

In [6]:
relevante = {}
irrelevante = {}

for palavra in tudo:
    relevante[palavra] = 1
    irrelevante[palavra] = 1
    
    
for i in range(len(data['Treinamento'])):
    a = data["Treinamento"][i].split(" ")
    if data['Relevância'][i] == 1:
        for palavra in a:
            relevante[palavra] += 1
    else:
        for palavra in a:
            irrelevante[palavra] += 1

In [7]:
prob_relevante = {}
prob_irrelevante = {}

for palavra in tudo:
    prob_relevante[palavra] = relevante[palavra] / (conta_relevante + len(tudo)) 
    prob_irrelevante[palavra] = irrelevante[palavra] / (conta_irrelevante + len(tudo)) 

In [8]:
tweets_rel = len(data[data["Relevância"] == 1])
tweets_irrel = len(data[data["Relevância"] == 0])

prob_tweets_irrel = tweets_irrel / (tweets_irrel + tweets_rel)
prob_tweets_rel = tweets_rel / (tweets_irrel + tweets_rel)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [9]:
df2 = pd.read_excel('tweets_netflix_teste.xlsx')

df2.replace(regex=['\n', '\t'], value = '', inplace = True) #corrigindo separação entre enter e tab - pt.1

In [10]:
tweets = []

remover = ['?', '!', ',', ';', '.', '”', '“', ')',
           '(', '*', '...', ':', '…', '{', '}', '"',
           '"', '=', "'", '•', '|', '-', '/'] #caracteres que devem ser removídos
for i in range(len(df2['Teste'])):
    m = ''
    tweet = df2['Teste'][i]
    for letra in tweet:
        if letra in UNICODE_EMOJI:
            m += ' ' + letra  + ' ' #corrigindo espaçamento entre caracteres - pt.2
        elif letra not in remover:
            m += letra
    tweets.append(m)

In [11]:
tweets_final = []
words = words = ['rt', 'a', 'is', 'me', 'in', 'netflix', 'as', 'it',
         'we', 'us', 'of', 'i', 'to', 'the', 'by', 'im', 'so',
         'are', 'or', 'has', 'have', 'their', 'that', 'else',
         'on', 'be', 'his', 'our', 'you', 'my', 'its', 'for',
         "it’s", 'this', 'if']
for i in tweets:
    post = i.split()
    palavras = ''
    for e in post:
        #removendo links, hashtags, @, e demais termos/palavras inúteis
        if e[0:4] != 'http' and e[0:1] != '@' and e[:1] != '#' and e not in words:
            palavras += '' + e + ' '
        elif e in UNICODE_EMOJI:
            palavras += '' + e + ''
            
    tweets_final.append(palavras)

In [12]:
data2 = pd.DataFrame(tweets_final, columns=['Teste'])
data2 = data2.join(df2['Relevância'], how='inner')

In [13]:
previsao = []

for tweet in data2['Teste']:
    prob_rel = 1
    prob_irrel = 1
    palavras = tweet.split(' ')
        
    for palavra in palavras:
        if palavra in prob_relevante:
            prob_rel *= prob_relevante[palavra]
        else:
            prob_rel *= (1 / (conta_relevante + len(tudo)))
            
    for palavra in palavras:   
        if palavra in prob_irrelevante:
            prob_irrel *= prob_irrelevante[palavra]
        else:
            prob_irrel *= (1 / (conta_irrelevante + len(tudo)))
            
    prob_rel *= prob_tweets_rel
    prob_irrel *= prob_tweets_irrel
        
    if prob_rel > prob_irrel:
        previsao.append(1)
    else:
        previsao.append(0)

In [14]:
data2['Previsão'] = previsao
data2

,Teste,Relevância,Previsão
0,week 3d film made with blender,0,0
1,got asked date and all want do stay bed and wa...,0,0
2,they need put,0,0
3,nearly headband still actually head strictly g...,0,1
4,mg aleatorio se lleva minecraft full accessopt...,0,0
5,should keep true source material because what ...,1,1
6,just wanna spend day cuddled up w boo watching...,0,0
7,gonna catch up new luke cage season &amp ozark...,0,0
8,yeah just finished and still wrong 22 doing ba...,0,1
9,black girls wanting swim but risking getting y...,0,0


## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


If categorical variable has a category (in test data set), which was not observed in training data set, then model will assign a 0 (zero) probability and will be unable to make a prediction. This is often known as “Zero Frequency”. To solve this, we can use the smoothing technique. One of the simplest smoothing techniques is called Laplace estimation.

On the other side naive Bayes is also known as a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.

Another limitation of Naive Bayes is the assumption of independent predictors. In real life, it is almost impossible that we get a set of predictors which are completely independent.

**Melhorias**

If continuous features do not have normal distribution, we should use transformation or different methods to convert it in normal distribution.
If test data set has zero frequency issue, apply smoothing techniques “Laplace Correction” to predict the class of test data set.
Remove correlated features, as the highly correlated features are voted twice in the model and it can lead to over inflating importance.
Naive Bayes classifiers has limited options for parameter tuning like alpha=1 for smoothing, fit_prior=[True|False] to learn class prior probabilities or not and some other options (look at detail here). I would recommend to focus on your  pre-processing of data and the feature selection.
You might think to apply some classifier combination technique like ensembling, bagging and boosting but these methods would not help. Actually, “ensembling, boosting, bagging” won’t help since their purpose is to reduce variance. Naive Bayes has no variance to minimize.